In [103]:
from pycorenlp import StanfordCoreNLP
import os
from multiprocessing import Pool
import time

In [104]:
nlp = StanfordCoreNLP('http://localhost:9000')

In [3]:
drug_name_file = open('drugs_com_web_names.txt','r')
drug_names = drug_name_file.readlines()
drug_names = [drug_name.replace('\n','') for drug_name in drug_names]

In [9]:
def extract_grams(drugname):
    save_filepath = 'drugs/grams/' + drugname + '.txt'
    if not os.path.exists(save_filepath):
        read_filepath = 'drugs/preprocessed/' + drugname + '.txt'
        file = open(read_filepath)
        texts = file.readlines()
        grams = []
        for text in texts:
            if len(text.split()) > 3:
                while True:
                    try:
                        output = nlp.annotate(text, properties={
                          'annotators': 'pos',
                          'outputFormat': 'json'
                          })
                        break
                    except:
                        print drugname + ': Failed in parsing...'
                        time.sleep(1)
                for sentence in output['sentences']:
                    tokens = sentence['tokens']
                    index = 0
                    while index < len(tokens):
                        if ('NN' in tokens[index]['pos']) or ('VBG' in tokens[index]['pos']):
                            gram = tokens[index]['word'].lower()
                            offset = -1
                            while (index + offset >= 0) and (('JJ' in tokens[index+offset]['pos']) or 
                                    ('VBG' in tokens[index+offset]['pos']) or ('VBN' in tokens[index+offset]['pos'])):
                                if ('included' in tokens[index+offset]['word'].lower()) or ('including' in tokens[index+offset]['word'].lower()):
                                    break
                                gram = tokens[index+offset]['word'].lower() + ' ' + gram
                                offset -= 1
                            index += 1
                            while (index < len(tokens)) and (('NN' in tokens[index]['pos']) or 
                                ('IN' in tokens[index]['pos'])):
                                if 'IN' in tokens[index]['pos']:
                                    if (index + 1 < len(tokens)) and (('JJ' in tokens[index+1]['pos']) 
                                        or('DT' in tokens[index+1]['pos']) or ('NN' in tokens[index+1]['pos'])):
                                        gram += ' ' + tokens[index]['word'].lower()
                                        index += 1
                                    else:
                                        break
                                    while (index < len(tokens)) and (('JJ' in tokens[index]['pos']) 
                                                                     or ('DT' in tokens[index]['pos'])):
                                        gram += ' ' + tokens[index]['word'].lower()
                                        index += 1
                                else:
                                    gram += ' ' + tokens[index]['word'].lower()
                                    index += 1
                            grams.append(gram)
                        index += 1
            else:
                grams.append(text.rstrip('\n').lower())

        if grams:
            with open(save_filepath, 'w') as file:
                file.write("\n".join(grams))
                file.close()

In [6]:
# pool = Pool()
# result = pool.map_async(extract_grams, drug_names, chunksize=int(len(drug_names)/8))
# while (True):
#   if (result.ready()): break
#   remaining = result._number_left
#   print "Waiting for", remaining, "tasks to complete..."
#   time.sleep(1)
# print 'All tasks completed!'
# pool.close()

In [11]:
extract_grams('a-hydrocort')

In [26]:
for drugname in drug_names:
    extract_grams(drugname)

In [48]:
for drugname in drug_names:
    filename = 'drugs/preprocessed/' + drugname + '.txt'
    if os.stat(filename).st_size == 0:
        print drugname

In [25]:
import re
# my_str = 'Nervous system side effects have included headache (16%), fatigue (5.6%), and somnolence (5% to 20).  Somnolence tends to be dose-related and generally occurs more frequently with doses higher than 10 mg per day.  Other nervous system side effects include dizziness (1.8), insomnia (1.5%), and nervousness (1.1%).  Post marketing reports have included aggression reaction and convulsions.'
my_str = 'Nervous system side effects have included headache (16%), fatigue (), and somnolence (5% to 20%).'
my_str = re.sub('\d+[.]*\d*[%]*', '', my_str)
my_str

'Nervous system side effects have included headache (), fatigue (), and somnolence ( to ).'

In [106]:
output = nlp.annotate('increased apetites', properties={
                          'annotators': 'lemma',
                          'outputFormat': 'json'
                          })
output

{u'sentences': [{u'index': 0,
   u'tokens': [{u'after': u' ',
     u'before': u'',
     u'characterOffsetBegin': 0,
     u'characterOffsetEnd': 9,
     u'index': 1,
     u'lemma': u'increase',
     u'originalText': u'increased',
     u'pos': u'VBN',
     u'word': u'increased'},
    {u'after': u'',
     u'before': u' ',
     u'characterOffsetBegin': 10,
     u'characterOffsetEnd': 18,
     u'index': 2,
     u'lemma': u'apetite',
     u'originalText': u'apetites',
     u'pos': u'NNS',
     u'word': u'apetites'}]}]}